# Assigment 5

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Generate a function called **import_data_sbs_banks** that take as arguments `start_month`, `start_year`, `end_month` and `end_year`. This function should be able to import the files in **..\_data\sbs\B_RawData\bancos** taking into consideration these arguments. Additionally, it should work if it is only given the first two arguments `(start_mont, start_year)`. The function should `returns` a nested dictionary. The `main key` must be year and `second key` must be `month`. The values should be the dataframes.

In [36]:
# Install the packages as required.
import os
import pandas as pd
from datetime import datetime

In [37]:
def import_data_sbs_banks(start_month, start_year, end_month = None, end_year = None):

    # Read all the files contained in the specified path and save them in a list
    path = r"../../_data\sbs\B_RawData\bancos"
    file_list = [os.path.join(path, file) for file in os.listdir(path)]
    
    # Create empty dictionary to save dataframes
    excel_dict = {} 
    
    # Iterate over each element in "file_list" 
    for file in file_list:
        # Read excel file and save in a provisional df
        df_tmp = pd.read_excel(file)
        # Create column "date_info" with the day information
        df_tmp["date_info"] = os.path.splitext(file)[0][45:]
        # Change the type of the "date_info" column to date 
        df_tmp["date_info"] = pd.to_datetime(df_tmp["date_info"], format = "%d_%m_%Y")
        # Save all files in the dictionary 
        excel_dict[file] = df_tmp     
    
    # In case we only define the values of start_month and start_year
    if end_month is None:
        end_month = start_month
    if end_year is None:
        end_year = start_year    
        
    # Create start_date and end_date by values of arguments
    # Use "pd.to_datetime" to transform string to date
    start_date = pd.to_datetime(f'{start_month}/01/{start_year}', format = '%m/%d/%Y')
    # Specify "end_month+1" in end_date so that the range covers the whole month
    # Add a condition for December, in that case, it is necessary to take as reference the first day of next year
    if end_month == 12:
        end_date = pd.to_datetime(f'01/01/{end_year + 1}', format = '%m/%d/%Y')
    else: 
        end_date = pd.to_datetime(f'{end_month + 1}/01/{end_year}', format = '%m/%d/%Y') 

    # Select dfs whose "date_info" column falls between start_date and end_date
    selected_dfs = {
        k: v for k, v in excel_dict.items() 
        if start_date <= pd.Timestamp(v['date_info'].iloc[0]) <= end_date
    }
    
    # Sort the values in selected_dfs by date
    sorted_dfs = sorted(selected_dfs.items(), key =lambda x: (x[1]['date_info'].dt.year.values[0],
                                                              x[1]['date_info'].dt.month.values[0]))
    
    # Create empty dictionary to store nested dataframes
    nested_dict = {}
    
    # Nest selected dataframes in the dictionary
    # Use the years as the first level keys and the months as the second level keys
    # The final dict will have the next structure: "nested_dict[year][month] = value"
    for key, value in sorted_dfs:
        year  = value['date_info'].dt.year.values[0]
        month = value['date_info'].dt.month.values[0]
        nested_dict.setdefault(year, {})[month] = value

    return nested_dict


In [38]:
import_data_sbs_banks(10, 2021, 2, 2022)

{2021: {10:                                        Tasa Anual (%)   BBVA Comercio Crédito  \
  0                                        Corporativos   1.96     3.95    2.31   
  1                                          Descuentos   3.16        -    3.33   
  2                             Préstamos hasta 30 días   1.58        -    2.03   
  3                           Préstamos de 31 a 90 días   2.85        -    2.49   
  4                          Préstamos de 91 a 180 días   1.93     3.95    1.98   
  5                         Préstamos de 181 a 360 días   1.71        -    1.89   
  6                         Préstamos a más de 360 días   2.64        -    2.78   
  7                                    Grandes Empresas   5.24     8.46    4.93   
  8                                          Descuentos   8.50     8.00    5.51   
  9                             Préstamos hasta 30 días   5.56        -    5.37   
  10                          Préstamos de 31 a 90 días   4.47     9.00    4.

### Test the extraction of the dataframes according to the keys

In [39]:
df_banks = import_data_sbs_banks(10, 2021, 2, 2022)
df_banks[2022][1]

,Tasa Anual (%),BBVA,Comercio,Crédito,Pichincha,BIF,Scotiabank,Citibank,Interbank,Mibanco,GNB,Falabella,Santander,Ripley,Alfin,ICBC,Bank of China,Promedio,date_info
0,Corporativos,6.09,5.50,3.67,6.30,4.05,4.24,3.83,4.14,-,4.91,-,5.96,-,-,4.99,-,4.18,2022-01-31
1,Descuentos,4.12,5.50,3.76,5.52,4.76,3.82,-,4.25,-,-,-,6.34,-,-,-,-,4.67,2022-01-31
2,Préstamos hasta 30 días,4.23,-,3.24,-,4.40,2.82,-,-,-,-,-,-,-,-,-,-,3.11,2022-01-31
3,Préstamos de 31 a 90 días,7.09,-,3.88,5.05,4.18,5.35,3.83,4.38,-,4.78,-,6.02,-,-,4.55,-,4.79,2022-01-31
4,Préstamos de 91 a 180 días,4.81,-,3.28,5.90,3.55,3.67,-,3.88,-,5.59,-,5.41,-,-,6.00,-,3.69,2022-01-31
5,Préstamos de 181 a 360 días,4.24,-,3.05,7.50,-,3.41,-,-,-,-,-,5.45,-,-,-,-,3.44,2022-01-31
6,Préstamos a más de 360 días,7.74,-,4.30,8.25,4.70,4.08,-,-,-,-,-,5.70,-,-,-,-,4.50,2022-01-31
7,Grandes Empresas,7.33,9.85,5.61,6.16,6.51,5.10,5.43,6.07,-,4.75,-,6.93,-,-,-,-,6.07,2022-01-31
8,Descuentos,9.47,-,5.98,6.36,6.34,4.94,-,6.60,-,-,-,6.66,-,-,-,-,6.82,2022-01-31
9,Préstamos hasta 30 días,7.53,-,5.08,5.20,5.25,5.33,-,4.48,-,4.43,-,8.15,-,-,-,-,5.37,2022-01-31


### Test function according to their keys

In [40]:
nested_dict = import_data_sbs_banks(12, 2021, 6, 2022)

for year in nested_dict.keys():
    for month in nested_dict[year].keys():
        print("Year: {}, Month: {}".format(year, month))

Year: 2021, Month: 12
Year: 2022, Month: 1
Year: 2022, Month: 2
Year: 2022, Month: 3
Year: 2022, Month: 4
Year: 2022, Month: 5
Year: 2022, Month: 6


In [41]:
nested_dict2 = import_data_sbs_banks(12, 2021)

for year in nested_dict2.keys():
    for month in nested_dict2[year].keys():
        print("Year: {}, Month: {}".format(year, month))

Year: 2021, Month: 12
